In [1]:
!pip install spacy
!pip install beautifulsoup4
!python -m spacy download pt
!python -m spacy download en

⚠ As of spaCy v3.0, shortcuts like 'pt' are deprecated. Please use the
full pipeline package name 'pt_core_news_sm' instead.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 76.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 73.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependenc

In [2]:
import nltk
import spacy
from spacy import displacy
import tqdm
import operator
import requests
from bs4 import BeautifulSoup
import re
import pt_core_news_sm
import en_core_web_sm
from collections import Counter
import random
import operator
import matplotlib.pyplot as plt

# Pos-tagging

Um dos objetivos do pos-tagging pode ser por exemplo identificar verbos para identificar as ações de um post.

Podemos usar duas abordagens para pos-tagging
* Regras (como por exemplo Regex)
* Modelos

Iremos trabalhar aqui com a primeira abordagem

In [3]:
def print_found_groups(pattern, phrase):
    regex = re.compile(pattern)
    for m in regex.finditer(phrase):
        print({k:v for(k,v) in m.groupdict().items() if v is not None})

In [4]:
phrase1 = "eu estudo, você estuda, nós estudamos e vc sabe como é... estudos em todo lugar "

In [5]:
estudar_pattern_try1 = r'(?P<VERB_ESTUDAR>\bestud\w?\b)' ##O primeiro padrão é para nomear o padrao identificado

In [6]:
print_found_groups(estudar_pattern_try1, phrase1)

{'VERB_ESTUDAR': 'estudo'}
{'VERB_ESTUDAR': 'estuda'}


In [7]:
##Pegando todas as conjugações
estudar_pattern_try2 = r'(?P<VERB_ESTUDAR>\bestud\w{1,4}\b)'
print_found_groups(estudar_pattern_try2, phrase1)

{'VERB_ESTUDAR': 'estudo'}
{'VERB_ESTUDAR': 'estuda'}
{'VERB_ESTUDAR': 'estudamos'}
{'VERB_ESTUDAR': 'estudos'}


In [8]:
##Pegando todos os verbos da seguinte frase
phrase2 = "O principal uso que fazemos do canvas é ver e entregar exercícios, além de acessar os materiais postados pelos professores. É importante realizar o acesso todos os dias."

def get_found_groups(pattern, phrase):
    regex = re.compile(pattern)
    return {v:k for m in regex.finditer(phrase) for(k,v) in m.groupdict().items() if v is not None }

verbs_list = ['uso', 'faz\w{0,4}','ver','entregar','acess\w{1,2}', 'realizar', 'é']
manyverbs_pattern_solution = r'(?P<VERB>'+"|".join(verbs_list)+')'

In [9]:
"|".join(verbs_list)

'uso|faz\\w{0,4}|ver|entregar|acess\\w{1,2}|realizar|é'

In [10]:
manyverbs_pattern_solution

'(?P<VERB>uso|faz\\w{0,4}|ver|entregar|acess\\w{1,2}|realizar|é)'

In [11]:
get_found_groups(manyverbs_pattern_solution, phrase2)

{'uso': 'VERB',
 'fazemos': 'VERB',
 'é': 'VERB',
 'ver': 'VERB',
 'entregar': 'VERB',
 'acessar': 'VERB',
 'realizar': 'VERB',
 'acesso': 'VERB'}

In [12]:
##Construindo regex para pegar substantivos da frase
subs_list = ['acesso','canvas', 'materia\w{0,2}', 'professor\w{0,2}', 'exercício\w{0,1}']
subs_pattern_solution = r'(?P<SUBS>'+"|".join(subs_list)+')'

In [13]:
get_found_groups(subs_pattern_solution, phrase2)

{'canvas': 'SUBS',
 'exercícios': 'SUBS',
 'materiais': 'SUBS',
 'professores': 'SUBS',
 'acesso': 'SUBS'}

In [14]:
##Marcando a classe gramatical nas frases
def get_postag(token, postag):
    return None if token not in list(postag.keys()) else postag[token]

def compose_postag(token, subs, verbs):
    s = get_postag(token, subs) #SUBS
    v = get_postag(token, verbs) #VERB
    if s is not None and v is not None: #Sim
        return token + '_' + s + '|' + v # acesso_SUBS|VERB
    if s is None and v is not None:
        return token + '_' + v
    if v is None and s is not None:
        return token + '_' + s
    return token

def transform(phrase):
    tokens = re.findall(r"\w+(?:'\w+)?|[^\w\s]", phrase)
    subs = get_found_groups(subs_pattern_solution, phrase)
    verbs = get_found_groups(manyverbs_pattern_solution, phrase)
    return ' '.join([compose_postag(token, subs, verbs) for token in tokens])

In [15]:
transform(phrase2)

'O principal uso_VERB que fazemos_VERB do canvas_SUBS é_VERB ver_VERB e entregar_VERB exercícios_SUBS , além de acessar_VERB os materiais_SUBS postados pelos professores_SUBS . É importante realizar_VERB o acesso_SUBS|VERB todos os dias .'

## MacMorpho NLTK

In [16]:
nltk.download('mac_morpho')
print("WORDS ", nltk.corpus.mac_morpho.tagged_words())
print("______________")
print("SENTS ",nltk.corpus.mac_morpho.tagged_sents())

[nltk_data] Downloading package mac_morpho to /root/nltk_data...
[nltk_data]   Unzipping corpora/mac_morpho.zip.


WORDS  [('Jersei', 'N'), ('atinge', 'V'), ('média', 'N'), ...]
______________
SENTS  [[('Jersei', 'N'), ('atinge', 'V'), ('média', 'N'), ('de', 'PREP'), ('Cr$', 'CUR'), ('1,4', 'NUM'), ('milhão', 'N'), ('em', 'PREP|+'), ('a', 'ART'), ('venda', 'N'), ('de', 'PREP|+'), ('a', 'ART'), ('Pinhal', 'NPROP'), ('em', 'PREP'), ('São', 'NPROP'), ('Paulo', 'NPROP')], [('Programe', 'V'), ('sua', 'PROADJ'), ('viagem', 'N'), ('a', 'PREP|+'), ('a', 'ART'), ('Exposição', 'NPROP'), ('Nacional', 'NPROP'), ('do', 'NPROP'), ('Zebu', 'NPROP'), (',', ','), ('que', 'PRO-KS-REL'), ('começa', 'V'), ('dia', 'N'), ('25', 'N|AP')], ...]


In [17]:
nltk.corpus.mac_morpho.tagged_sents()[0]

[('Jersei', 'N'),
 ('atinge', 'V'),
 ('média', 'N'),
 ('de', 'PREP'),
 ('Cr$', 'CUR'),
 ('1,4', 'NUM'),
 ('milhão', 'N'),
 ('em', 'PREP|+'),
 ('a', 'ART'),
 ('venda', 'N'),
 ('de', 'PREP|+'),
 ('a', 'ART'),
 ('Pinhal', 'NPROP'),
 ('em', 'PREP'),
 ('São', 'NPROP'),
 ('Paulo', 'NPROP')]

In [18]:
words_tagged = nltk.corpus.mac_morpho.tagged_words()
tags = {}
for word, tag in words_tagged:
    if tag not in tags.keys():
        tags[tag] = 1
    else:
        tags[tag] += 1

tags_sorted = sorted(tags.items(), key=operator.itemgetter(1)) ##ordendando as tags
dict(tags_sorted[-5:])

{'V': 98056, 'PREP': 104364, 'NPROP': 114318, 'ART': 151891, 'N': 236462}

In [19]:
dict_tag = {}

for word, tag in nltk.corpus.mac_morpho.tagged_words():
  if word in dict_tag.keys():
    dict_tag[word] = list(set(dict_tag[word] + [tag]))
  else:
    dict_tag[word] = [tag]

In [20]:
dict_tag['São']

['PDEN', 'NPROP', 'VAUX', 'N', 'V']

## PosTagging com Spacy

In [21]:
nlp = spacy.load("pt_core_news_sm")
def analisa_text(txt):
    doc = nlp(txt)
    for tok in doc:
        print("Token: ", tok.text, "\t\ta) Tag: ",tok.pos_,"\t\tb) Dep:", tok.dep_)
    print("c)")
    displacy.render(doc, style='dep',jupyter=True)
txt0="Ele foi andar"
txt1="O andar estava triste"

In [22]:
analisa_text(txt0)
analisa_text(txt1)

Token:  Ele 		a) Tag:  PRON 		b) Dep: nsubj
Token:  foi 		a) Tag:  AUX 		b) Dep: cop
Token:  andar 		a) Tag:  VERB 		b) Dep: ROOT
c)


Token:  O 		a) Tag:  DET 		b) Dep: det
Token:  andar 		a) Tag:  NOUN 		b) Dep: nsubj
Token:  estava 		a) Tag:  AUX 		b) Dep: cop
Token:  triste 		a) Tag:  VERB 		b) Dep: ROOT
c)


# Reconhecimento de entidades nomeadas

Iremos capturar o texto das noticias abaixo
Capture o texto de cada notícia abaixo:

- 0 -> https://g1.globo.com/sp/sao-paulo/noticia/2020/03/03/cai-para-130-numero-de-casos-suspeitos-de-coronavirus-no-estado-de-sao-paulo.ghtml
- 1 -> https://g1.globo.com/sp/sao-paulo/noticia/2020/03/04/ministerio-da-saude-confirma-terceiro-paciente-com-coronavirus-em-sp-um-quarto-caso-aguarda-contraprova.ghtml
- 2 -> https://g1.globo.com/bemestar/coronavirus/noticia/2020/03/04/ultimas-noticias-de-coronavirus-de-4-de-marco.ghtml
- 3 -> https://g1.globo.com/economia/noticia/2020/03/04/projetos-reduzem-em-r-96-bilhoes-emendas-parlamentares-no-orcamento-2020-diz-governo.ghtml
- 4 -> https://g1.globo.com/politica/noticia/2020/03/04/documento-liga-assessor-de-eduardo-bolsonaro-a-conta-em-rede-social-para-ataques-pessoais.ghtml
- 5 -> https://g1.globo.com/rj/rio-de-janeiro/noticia/2020/03/04/flavio-bolsonaro-entra-com-recurso-para-tentar-paralisar-investigacao-sobre-suspeita-de-rachadinha.ghtml
- 6 -> https://g1.globo.com/economia/noticia/2020/03/04/pib-do-brasil-cresce-11percent-em-2019.ghtml
- 7 -> https://g1.globo.com/economia/noticia/2020/03/04/tudo-dentro-do-previsto-diz-paulo-guedes-sobre-resultado-do-pib.ghtml
- 8 -> https://g1.globo.com/sp/santos-regiao/noticia/2020/03/04/sobe-numero-de-mortos-apos-temporal-na-baixada-santista.ghtml
- 9 -> https://g1.globo.com/sp/sao-paulo/noticia/2020/03/04/ministerio-da-saude-confirma-terceiro-paciente-com-coronavirus-em-sp-um-quarto-caso-aguarda-contraprova.ghtml

In [23]:
def match_class(target):
    def do_match(tag):
        classes = tag.get('class', [])
        return all(c in classes for c in target)
    return do_match

def get_text_url(url):
    res = requests.get(url)
    html = res.text
    soup = BeautifulSoup(html, 'html5lib')
    #remove marcações de scripts e style
    texto = soup.find_all(match_class(["content-text__container"]))
    all_text = ""
    for t in texto:
        all_text += t.get_text()
    return all_text

In [24]:
noticias = {}
noticias[0] = "https://g1.globo.com/sp/sao-paulo/noticia/2020/03/03/cai-para-130-numero-de-casos-suspeitos-de-coronavirus-no-estado-de-sao-paulo.ghtml"
noticias[1] = "https://g1.globo.com/sp/sao-paulo/noticia/2020/03/04/ministerio-da-saude-confirma-terceiro-paciente-com-coronavirus-em-sp-um-quarto-caso-aguarda-contraprova.ghtml"
noticias[2] = "https://g1.globo.com/bemestar/coronavirus/noticia/2020/03/04/ultimas-noticias-de-coronavirus-de-4-de-marco.ghtml"
noticias[3] = "https://g1.globo.com/economia/noticia/2020/03/04/projetos-reduzem-em-r-96-bilhoes-emendas-parlamentares-no-orcamento-2020-diz-governo.ghtml"
noticias[4] = "https://g1.globo.com/politica/noticia/2020/03/04/documento-liga-assessor-de-eduardo-bolsonaro-a-conta-em-rede-social-para-ataques-pessoais.ghtml"
noticias[5] = "https://g1.globo.com/rj/rio-de-janeiro/noticia/2020/03/04/flavio-bolsonaro-entra-com-recurso-para-tentar-paralisar-investigacao-sobre-suspeita-de-rachadinha.ghtml"
noticias[6] = "https://g1.globo.com/economia/noticia/2020/03/04/pib-do-brasil-cresce-11percent-em-2019.ghtml"
noticias[7] = "https://g1.globo.com/economia/noticia/2020/03/04/tudo-dentro-do-previsto-diz-paulo-guedes-sobre-resultado-do-pib.ghtml"
noticias[8] = "https://g1.globo.com/sp/santos-regiao/noticia/2020/03/04/sobe-numero-de-mortos-apos-temporal-na-baixada-santista.ghtml"
noticias[9] = "https://g1.globo.com/sp/sao-paulo/noticia/2020/03/04/ministerio-da-saude-confirma-terceiro-paciente-com-coronavirus-em-sp-um-quarto-caso-aguarda-contraprova.ghtml"

In [25]:
texto_noticias = {i:get_text_url(noticias[i]) for i in noticias.keys()}

In [26]:
texto_noticias[0]

' A Secretaria Estadual de Saúde de São Paulo divulgou nesta terça-feira (3) que caiu o número de pacientes com suspeita de coronavírus no estado. Enquanto na segunda-feira (2) eram 163 casos suspeitos, nesta terça são 130.  Foram descartados, ao todo, 104 casos no estado, destes, 56 foram nas últimas 24 horas. Entretanto, segundo informou a pasta, entre segunda (2) e terça (3), 23 novos surgiram.  Permanecem confirmados apenas 2 casos da doença no país, ambos no estado de São Paulo. Os pacientes estão em quarentena domiciliar. Ambos estiveram em viagem na Itália.  "Os dois confirmados estão evoluindo bem clinicamente, mas ainda têm alguns sintomas que fazem com que eles continuem em isolamento domiciliar", salientou Paulo Menezes, coordenador do comitê de operações emergenciais (COE) da Secretaria Estadual de Saúde.  O número de pessoas que tiveram contato com o segundo caso confirmado não foi divulgado pela secretaria. No primeiro caso confirmado, eram 34 pessoas, entre passageiros d

In [27]:
##Utilizando o Spacy
nlp = pt_core_news_sm.load()
nlp_texto_noticias = {i:nlp(texto_noticias[i]) for i in texto_noticias.keys()}
entidades = {i:set(map(str, nlp_texto_noticias[i].ents)) for i in nlp_texto_noticias.keys()}

In [28]:
nlp_texto_noticias[0].ents

(Secretaria Estadual de Saúde de São Paulo,
 estado de São Paulo,
 Itália,
 Paulo Menezes,
 Secretaria Estadual de Saúde,
 Paulo Menezes,
 José Henrique Germann,
 Centro de Contingenciamento de Emergências,
 David Uip,
 José Henrique Germann,
 Helena Sato,
 Centro de Vigilância Epidemiológica,
 CVE,
 Uip,
 Adolfo Lutz,
 Menezes,
 Secretaria Estadual de Saúde,
 Secretaria Estadual de Saúde,
 Ministério da Saúde,
 coronavírus,
 São Paulo,
 Milão,
 Itália,
 Menezes,
 Saúde,
 Uip,
 Organização Mundial de Saúde,
 OMS,
 R$,
 Covid-19,
 David Uip,
 Instituto Butantan,
 Secretário de Estado da Saúde,
 José Henrique Germann,
 Campanha Nacional de Vacinação,
 H1N1,
 Covid-19)

In [29]:
entidades

{0: {'Adolfo Lutz',
  'CVE',
  'Campanha Nacional de Vacinação',
  'Centro de Contingenciamento de Emergências',
  'Centro de Vigilância Epidemiológica',
  'Covid-19',
  'David Uip',
  'H1N1',
  'Helena Sato',
  'Instituto Butantan',
  'Itália',
  'José Henrique Germann',
  'Menezes',
  'Milão',
  'Ministério da Saúde',
  'OMS',
  'Organização Mundial de Saúde',
  'Paulo Menezes',
  'R$',
  'Saúde',
  'Secretaria Estadual de Saúde',
  'Secretaria Estadual de Saúde de São Paulo',
  'Secretário de Estado da Saúde',
  'São Paulo',
  'Uip',
  'coronavírus',
  'estado de São Paulo'},
 1: {'Adolfo Lutz',
  'Alemanha',
  'Brasil',
  'COVID-19',
  'Campanha Nacional de Vacinação',
  'Centro de Contingência',
  'Centro de Contingência de São Paulo',
  'Colômbia',
  'Covid-19',
  'David Uip',
  'Einstein',
  'Espanha',
  'Europa',
  'H1N1',
  'Hospital Albert Einstein',
  'Hospital Beneficência Portuguesa',
  'Instituto Adolfo Lutz',
  'Itália',
  'João Doria',
  'Menezes',
  'Milão',
  'Ministé

In [30]:
all_entidades = []
for k in entidades.keys():
    all_entidades.extend(entidades[k])
all_entidades = dict(Counter(all_entidades))
all_entidades_sorted = sorted(all_entidades.items(), key=operator.itemgetter(1))
all_entidades_sorted

[('Helena Sato', 1),
 ('CVE', 1),
 ('Centro de Contingenciamento de Emergências', 1),
 ('Centro de Vigilância Epidemiológica', 1),
 ('Secretário de Estado da Saúde', 1),
 ('José Henrique Germann', 1),
 ('Instituto Butantan', 1),
 ('Secretaria Estadual de Saúde de São Paulo', 1),
 ('Arábia Saudita', 1),
 ('Meca', 1),
 ('Destaques', 1),
 ('Irã', 1),
 ('DEM-AP', 1),
 ('Amarela', 1),
 ('Programa Verde', 1),
 ('Davi Alcolumbre', 1),
 ('Otavio Ladeira', 1),
 ('MP Verde', 1),
 ('Ladeira', 1),
 ('Legislativo', 1),
 ('Amarelo', 1),
 ('Congresso Nacional', 1),
 ('Orçamento de 2020', 1),
 ('Tesouro', 1),
 ('Esteves Colnago', 1),
 ('Colnago', 1),
 ('Orçamento 2020', 1),
 ('Tesouro Nacional', 1),
 ('Bolsofeios', 1),
 ('Fake News', 1),
 ('Joice', 1),
 ('Mandar', 1),
 ('Jornal Nacional', 1),
 ('Câmara dos Deputados', 1),
 ('Olha', 1),
 ('Comissão Parlamentar Mista de Inquérito', 1),
 ('Guimarães', 1),
 ('Eduardo Bolsonaro', 1),
 ('UOL', 1),
 ('Conselho de Ética da Câmara dos Deputados', 1),
 ('Túlio 

In [31]:
nlp_texto_noticias[0].ents[-9]

R$

In [32]:
nlp_texto_noticias[0].ents[-9].label_

'LOC'

In [33]:
## Obtendo os rotulos
labels = []
for i in list(nlp_texto_noticias.keys()):
    labels.extend([x.label_ for x in nlp_texto_noticias[i].ents])
labels = Counter(labels)
len(labels)

4

In [34]:
labels

Counter({'MISC': 49, 'LOC': 211, 'PER': 72, 'ORG': 30})

In [35]:
spacy.explain('MISC')

'Miscellaneous entities, e.g. events, nationalities, products or works of art'

In [36]:
spacy.explain('LOC')

'Non-GPE locations, mountain ranges, bodies of water'

In [37]:
spacy.explain('PER')

'Named person or family.'

In [38]:
spacy.explain('ORG')

'Companies, agencies, institutions, etc.'

In [39]:
displacy.render(nlp_texto_noticias[0], jupyter=True, style='ent')

## Retreino

In [40]:
#dados de treino. Note que para cada frase, a posição de uma determinada entidade.
train_data = [
("Em 04/12/1992 nasceu Buma", [(3, 13, "DATE"), (21,25, "PER")]),
("Data de início: 10/01/2018", [(16, 26, "DATE")]),
("Buma se mudou para Paris", [(0,4,"PER"), (20,26, "LOC")]),
("Paris cidade das luzes", [(0,5, "LOC")]),
("Buma nasceu em Contagem no dia 07/05/2018", [(0,4,"PER"),(15,23,"LOC"),(31,41, "DATE")])
]

In [41]:
nlp = spacy.load("pt_core_news_sm")
texto_nlp = nlp('Buma nasceu em Paris em 04/05/1995')
displacy.render(texto_nlp, style='ent', jupyter=True)

In [42]:
from spacy.training import Example
def train_spacy(data, iterations):
    TRAIN_DATA = data
    nlp = spacy.load("pt_core_news_sm")
    disabled_pipes = []
    for pipe_name in nlp.pipe_names:
        if pipe_name != 'ner':
            nlp.disable_pipes(pipe_name)
            disabled_pipes.append(pipe_name)

    # get names of other pipes to disable them during training
    optimizer = nlp.create_optimizer()
    for _ in range(25):
        random.shuffle(TRAIN_DATA)
        for raw_text, entity_offsets in TRAIN_DATA:
            doc = nlp.make_doc(raw_text)
            example = Example.from_dict(doc, {"entities": entity_offsets})
            nlp.update([example], sgd=optimizer)

    for pipe_name in disabled_pipes:
        nlp.enable_pipe(pipe_name)
    return nlp

In [43]:
modelo_novo = train_spacy(train_data, 30)

/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Buma se mudou para Paris" with entities "[(0, 4, 'PER'), (20, 26, 'LOC')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


In [44]:
texto_nlp_novo = modelo_novo('Buma nasceu em Paris em 04/05/1995')

In [45]:
displacy.render(texto_nlp_novo, style='ent', jupyter=True)